<a href="https://colab.research.google.com/github/Ahnkyuwon504/AI-modeling/blob/main/hugging_face/2_quantize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch transformers accelerate huggingface_hub[cli] bitsandbytes

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 29.7 MB/s eta 0:00:0000:0100:01


In [ ]:
import torch

if torch.cuda.is_available():
    device = torch.device("cuda")  # CUDA 장치로 설정
else:
    device = torch.device("cpu")   # CPU 장치로 설정

device

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "42dot/42dot_LLM-SFT-1.3B"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map=device)

In [ ]:
!nvidia-smi

In [ ]:
text = '근육이 커지기 위해서는'
input_ids = tokenizer.encode(text, return_tensors='pt').to(device)
gen_ids = model.generate(input_ids,
                           max_length=256,
                           repetition_penalty=2.0,
                           pad_token_id=tokenizer.pad_token_id,
                           eos_token_id=tokenizer.eos_token_id,
                           bos_token_id=tokenizer.bos_token_id,
                           use_cache=True)
generated = tokenizer.decode(gen_ids[0])
print(generated)

In [ ]:
from transformers import pipeline

generator = pipeline(
    'text-generation',
    model=model,
    tokenizer=tokenizer

)
def query(instruction, input=None):
    if input:
        prompt = f"<human>: {instruction}\n\n<input>: {input}\n\n<bot>:"
    else:
        prompt = f"<human>: {instruction}\n\n<bot>:"

    print(generator(
        prompt,
        do_sample=True,
        top_p=0.9,
        early_stopping=True,
        max_length=512,
    )[0]['generated_text'])


query("3박 4일 하노이 여행을 가려는데 일정을 짜줘.")

In [ ]:
from transformers import TextStreamer, GenerationConfig

streamer = TextStreamer(tokenizer)
def gen(device, instruction, input=''):
    generation_config = GenerationConfig(
        temperature = 0.1,
        max_new_tokens = 512,
        exponential_decay_length_penalty = (512, 1.03),
        eos_token_id = tokenizer.eos_token_id,
        repetition_penalty = 1.05,
        do_sample = True,
        top_p = 0.7,
        min_length = 5,
        use_cache = True,
        return_dict_in_generate = True,
    )
    if input:
        prompt = f"<human>: {instruction}\n\n<input>: {input}\n\n<bot>:"
    else:
        prompt = f"<human>: {instruction}\n\n<bot>:"

    gened = model.generate(
        **tokenizer(
            prompt,
            return_tensors='pt',
            return_token_type_ids=False
        ).to(device),
        generation_config=generation_config,
        streamer=streamer
    )

In [ ]:
gen(device, "세종대왕에 대해서 알려줘.")